In [1]:
import keras.backend as K
from keras.layers import Input, Add, Dense, Activation, Concatenate, merge, Flatten, Dropout, Conv3D, MaxPooling3D, ZeroPadding3D, AveragePooling3D, TimeDistributed
from keras.models import Model
from keras.regularizers import l1, l2
from keras.optimizers import Adam, SGD
from keras.constraints import max_norm
from keras.layers.noise import GaussianNoise
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
import operator
import matplotlib.pyplot as plt
import config
import importlib
import math
import os
import copy
import pandas as pd
import re
import numpy as np
import helper_fxns as hf
import transforms as tr
from scipy.misc import imsave
import tensorflow as tf
%matplotlib inline

Using TensorFlow backend.


from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

nb_classes = C.nb_classes

X_train = np.expand_dims(x_train,axis=3)
X_train = np.expand_dims(X_train,axis=4)
X_test = np.expand_dims(x_test,axis=3)
X_test = np.expand_dims(X_test,axis=4)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [2]:
importlib.reload(config)
C = config.Config()

In [3]:
classes_to_include = ['cyst', 'hcc', 'hemangioma', 'cholangio']#, 'fnh', 'colorectal']
dims = C.dims
C.nb_classes = len(classes_to_include)

voi_img = Input(shape=(dims[0], dims[1], dims[2], C.nb_channels))
x = voi_img
#x = GaussianNoise(1)(x)
x = ZeroPadding3D(padding=(3,3,2))(voi_img)
x = Conv3D(filters=64, kernel_size=(3,3,2), activation='relu', kernel_constraint=max_norm(4.))(x)
x = Dropout(0.5)(x)
#x = MaxPooling3D((2, 2, 1))(x)
x = Conv3D(filters=128, kernel_size=(3,3,2), activation='relu', kernel_constraint=max_norm(4.))(x)
x = Dropout(0.5)(x)
#x = Conv3D(filters=128, kernel_size=(3,3,2), activation='relu', kernel_constraint=max_norm(4.))(x)
#x = Dropout(0.5)(x)
x = Conv3D(filters=64, kernel_size=(3,3,2), strides=(2, 2, 2), activation='relu', kernel_constraint=max_norm(4.))(x)
x = Dropout(0.5)(x)
x = Flatten()(x)

img_traits = Input(shape=(3,)) #size of lesion

big_x = Concatenate(axis=1)([x, img_traits])
big_x = Dense(128, activation='relu', kernel_constraint=max_norm(4.))(big_x)#, kernel_initializer='normal', kernel_regularizer=l1(.01), kernel_constraint=max_norm(3.))(x)
big_x = Dropout(0.5)(big_x)
pred_class = Dense(C.nb_classes, activation='softmax')(big_x)#Dense(C.nb_classes, activation='softmax')(x)

In [4]:
optim = Adam(lr=0.01)#, decay=0.001)
early_stopping = EarlyStopping(min_delta=0.001, patience=10)

model = Model([voi_img, img_traits], pred_class)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 24, 24, 12, 3) 0                                            
____________________________________________________________________________________________________
zero_padding3d_1 (ZeroPadding3D) (None, 30, 30, 16, 3) 0           input_1[0][0]                    
____________________________________________________________________________________________________
conv3d_1 (Conv3D)                (None, 28, 28, 15, 64 3520        zero_padding3d_1[0][0]           
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 28, 28, 15, 64 0           conv3d_1[0][0]                   
___________________________________________________________________________________________

model2 = Model(voi_img, intermed)

for l in range(2,len(model2.layers)):
    model2.layers[l].set_weights(model.layers[l].get_weights())

In [12]:
orig_data_dict = {}
num_samples = {}
voi_df = pd.read_csv(C.art_voi_path)
intensity_df = pd.read_csv(C.int_df_path)

def collect_unaug_data(classes_to_include, C):
    for class_name in classes_to_include:
        x = np.empty((10000, C.dims[0], C.dims[1], C.dims[2], C.nb_channels))
        x2 = np.empty((10000, 3))
        z = []

        for index, img_fn in enumerate(os.listdir(C.orig_dir+class_name)):
            try:
                x[index] = np.load(C.orig_dir+class_name+"\\"+img_fn)
            except:
                raise ValueError(C.orig_dir+class_name+"\\"+img_fn + " not found")
            z.append(img_fn)
            
            row = voi_df[(voi_df["Filename"] == img_fn[:img_fn.find('_')] + ".npy") &
                         (voi_df["lesion_num"] == int(img_fn[img_fn.find('_')+1:-4]))]
            x2[index] = [float(row["real_dx"]), float(row["real_dy"]), float(row["real_dz"])]
            
            x[index] = x[index] / float(intensity_df[intensity_df["AccNum"] == img_fn[:img_fn.find('_')]]["par_intensity"])

        x.resize((index, dims[0], dims[1], dims[2], C.nb_channels)) #shrink first dimension to fit
        x2.resize((index, 3)) #shrink first dimension to fit
        orig_data_dict[class_name] = [x,x2,np.array(z)]
        num_samples[class_name] = index
        
    return orig_data_dict, num_samples

In [13]:
orig_data_dict, num_samples = collect_unaug_data(classes_to_include, C)
print(num_samples)

{'cholangio': 58, 'cyst': 66, 'hcc': 60, 'hemangioma': 60}


In [14]:
train_ids = {} #filenames of training set originals
test_ids = {} #filenames of test set
X_test = []
X2_test = []
Y_test = []
Z_test = []
X_train_orig = []
X2_train_orig = []
Y_train_orig = []
Z_train_orig = []
cls_mapping = []

train_samples = {}

for cls_num, cls in enumerate(orig_data_dict):
    cls_mapping.append(cls)
    
    train_samples[cls] = round(num_samples[cls]*C.train_frac[cls])
    
    order = np.random.permutation(list(range(num_samples[cls])))
    train_ids[cls] = list(orig_data_dict[cls][2][order[:train_samples[cls]]])
    test_ids[cls] = list(orig_data_dict[cls][2][order[train_samples[cls]:]])
    
    X_test = X_test + list(orig_data_dict[cls][0][order[train_samples[cls]:]])
    X2_test = X2_test + list(orig_data_dict[cls][1][order[train_samples[cls]:]])
    Y_test = Y_test + [[0] * cls_num + [1] + [0] * (C.nb_classes - cls_num - 1)] * \
                        (num_samples[cls] - train_samples[cls])
    Z_test = Z_test + test_ids[cls]
    
    X_train_orig = X_train_orig + list(orig_data_dict[cls][0][order[:train_samples[cls]]])
    X2_train_orig = X2_train_orig + list(orig_data_dict[cls][1][order[:train_samples[cls]]])
    Y_train_orig = Y_train_orig + [[0] * cls_num + [1] + [0] * (C.nb_classes - cls_num - 1)] * \
                        (train_samples[cls])
    Z_train_orig = Z_train_orig + train_ids[cls]
    
    print("%s has %d samples for training (%d after augmentation) and %d for testing" %
          (cls, train_samples[cls], train_samples[cls] * C.aug_factor, num_samples[cls] - train_samples[cls]))
    
#Y_test = np_utils.to_categorical(Y_test, C.nb_classes)
#Y_train_orig = np_utils.to_categorical(Y_train_orig, C.nb_classes)

cholangio has 46 samples for training (2300 after augmentation) and 12 for testing
cyst has 53 samples for training (2650 after augmentation) and 13 for testing
hcc has 48 samples for training (2400 after augmentation) and 12 for testing
hemangioma has 48 samples for training (2400 after augmentation) and 12 for testing


In [15]:
aug_data_dict = {}

for class_name in classes_to_include:#os.listdir(C.aug_dir):
    x_train = np.empty((10000, dims[0], dims[1], dims[2], C.nb_channels))
    x2_train = np.empty((10000, 3))
    x_val = np.empty((10000, dims[0], dims[1], dims[2], C.nb_channels))
    x2_val = np.empty((10000, 3))
    z = []
    
    train_cnt = 0
    val_cnt = 0
    for img_fn in os.listdir(C.aug_dir+class_name):
        
        # part of training set
        if img_fn[:img_fn.rfind('_')] + ".npy" in train_ids[class_name]:
            try:
                x_train[train_cnt] = np.load(C.aug_dir+class_name+"\\"+img_fn)
            except:
                continue
                #raise ValueError(C.aug_dir+class_name+"\\"+img_fn + " cannot be loaded")
            train_cnt += 1
            z.append(img_fn)
            
            row = voi_df[(voi_df["Filename"] == img_fn[:img_fn.find('_')] + ".npy") &
                         (voi_df["lesion_num"] == int(img_fn[img_fn.find('_')+1:img_fn.rfind('_')]))]
            x2_train[train_cnt] = [float(row["real_dx"]), float(row["real_dy"]), float(row["real_dz"])]
            
        else:
            try:
                x_val[val_cnt] = np.load(C.aug_dir+class_name+"\\"+img_fn)
            except:
                continue
                #raise ValueError(C.aug_dir+class_name+"\\"+img_fn + " cannot be loaded")
            val_cnt += 1
            
            row = voi_df[(voi_df["Filename"] == img_fn[:img_fn.find('_')] + ".npy") &
                         (voi_df["lesion_num"] == int(img_fn[img_fn.find('_')+1:img_fn.rfind('_')]))]
            x2_val[val_cnt] = [float(row["real_dx"]), float(row["real_dy"]), float(row["real_dz"])]
    
    x_train.resize((train_cnt, dims[0], dims[1], dims[2], C.nb_channels)) #shrink first dimension to fit
    x2_train.resize((train_cnt, 3)) #shrink first dimension to fit
    x_val.resize((val_cnt, dims[0], dims[1], dims[2], C.nb_channels)) #shrink first dimension to fit
    x2_val.resize((val_cnt, 3)) #shrink first dimension to fit
    aug_data_dict[class_name] = {'train': [x_train, x2_train, np.array(z)], 'val': [x_val, x2_val]}

In [26]:
X_train = []
X2_train = []
Y_train = []
Z_train = []
X_val = []
X2_val = []
Y_val = []
Z_val = []

for cls_num, cls in enumerate(aug_data_dict):
    X_train = X_train + list(aug_data_dict[cls]['train'][0])
    X2_train = X2_train + list(aug_data_dict[cls]['train'][1])
    Y_train = Y_train + [[0] * cls_num + [1] + [0] * (C.nb_classes - cls_num - 1)] * len(aug_data_dict[cls]['train'][0])
    Z_train = Z_train + list(aug_data_dict[cls]['train'][2])
    
    X_val = X_val + list(aug_data_dict[cls]['val'][0])
    X2_val = X2_val + list(aug_data_dict[cls]['val'][1])
    Y_val = Y_val + [[0] * cls_num + [1] + [0] * (C.nb_classes - cls_num - 1)] * len(aug_data_dict[cls]['val'][0])

Z_cyst = data_dict["cyst"][1]

Y_[-4]

Z_cyst[-4]

X_cyst = data_dict["cyst"][0]
X_cyst = np.array(X_cyst) # X[:total_size//2]
X_cyst /= 255

X_hcc = data_dict["hcc"][0]
X_hcc = np.array(X_hcc) # X[:total_size//2]
X_hcc /= 255

X_train = delete_channel(X_train)
X_val = delete_channel(X_val)
X_test = delete_channel(X_test)
X_train_orig = delete_channel(X_train_orig)

def delete_channel(X):
    return np.expand_dims(np.array(X)[:,:,:,:,0], axis=4)

In [ ]:
#X = np.array([z for z in x])
#Y = np.array(y)
#Y = np.array([[0,1] if y[x] == 1 else [1,0] for x in range(len(y))])
#Y = K.constant(y, dtype=tf.int32)
#Y = K.one_hot(Y, C.nb_classes)

#total_size = X.shape[0]

#order = np.random.permutation(list(range(total_size)))
#X = X[order]
#Y = Y[order]

In [17]:
def scale_to_one(X, scale_by_channel=True):
    X = np.array(X)
    def scale_img(x):
        return 2 * (x - np.amin(x)) / (np.amax(x) - np.amin(x)) - 1
    
    if scale_by_channel:
        X = [np.stack([scale_img(x[:,:,:,0]), scale_img(x[:,:,:,1]), scale_img(x[:,:,:,2])], axis=3) for x in X]
    else:
        X = [scale_img(x) for x in X]
    
    return np.array(X)

In [18]:
#scale from -1 to 1
X_train = [scale_to_one(X_train), np.array(X2_train)]
X_val = [scale_to_one(X_val), np.array(X2_val)]
X_test = [scale_to_one(X_test), np.array(X2_test)]
X_train_orig = [scale_to_one(X_train_orig), np.array(X2_train_orig)]

Y_train = np.array(Y_train)
Y_val = np.array(Y_val)
Y_test = np.array(Y_test)
Y_train_orig = np.array(Y_train_orig)

Z_train = np.array(Z_train)
Z_test = np.array(Z_test)
Z_train_orig = np.array(Z_train_orig)

In [ ]:
hist = model.fit(X_train, Y_train, batch_size=32, epochs=200, validation_data=(X_val, Y_val)) #callbacks=[early_stopping]

Train on 9731 samples, validate on 2650 samples
Epoch 1/200
 480/9731 [>.............................] - ETA: 123s - loss: 1.3855 - acc: 0.3354- ETA: 203s - loss: 1.4873 - acc

def train_generator(X, Y):
    while True:
        for i in range(len(X)):
            yield np.expand_dims(X[i], axis=0), np.expand_dims(Y[i], axis=0)

num_epochs = 10
epoch_length = 100
best_loss = np.Inf
losses = np.zeros(epoch_length)
acc = np.zeros(epoch_length)

data_gen_train = train_generator(X_train, Y_train)
for epoch_num in range(num_epochs):
    print('Epoch {}/{}'.format(epoch_num + 1, num_epochs))
    iter_num = 0
    
    while True:
        X, Y = next(data_gen_train)
        losses[iter_num], acc[iter_num] = model.train_on_batch(X, Y)

        iter_num += 1
        if iter_num == epoch_length:
            curr_loss = np.mean(losses)
            curr_acc = np.mean(acc)
            print("Mean Loss:", curr_loss, "// Mean Accuracy:", curr_acc)

            if curr_loss < best_loss:
                print('Total loss decreased from {} to {}, saving weights'.format(best_loss,curr_loss))
                best_loss = curr_loss
                model.save_weights(C.model_path)
            break

In [200]:
plot_2ch_with_bbox("E100738220_0.npy", "idk", pad=30, draw_bbox=True, save_dir="classified_imgs")

(293, 293, 75, 2)
(213, 213, 55, 2)


In [19]:
voi_df_art = pd.read_csv(C.art_voi_path)
voi_df_ven = pd.read_csv(C.ven_voi_path)
voi_df_eq = pd.read_csv(C.eq_voi_path)

In [20]:
def plot_multich_with_bbox(fn, wrong_class, num_ch, pad=15, draw_bbox=True, save_dir=None):
    z = re.search(r'^(.*)_(\d*)\.', fn)
    voi_df_ven["id"] = voi_df_ven["id"].astype(str)
    
    img_fn = z.group(1) + ".npy"
    voi = voi_df_art[voi_df_art["Filename"] == img_fn].iloc[int(z.group(2))]
    
    img = np.load(C.full_img_dir+"\\"+img_fn)
    ven_voi = voi_df_ven[voi_df_ven["id"] == str(voi["id"])]
    if len(ven_voi) > 0:
        img = hf.align(img, voi, ven_voi.iloc[0], 1)
        
    if num_ch >= 3:
        eq_voi = voi_df_eq[voi_df_eq["id"] == str(voi["id"])]
        if len(eq_voi) > 0:
            img = hf.align(img, voi, eq_voi.iloc[0], 2)
        
    crop_size = C.dims
    
    #pad_img, x1, x2, y1, y2, z1, z2 = get_
    exceed_ratio = max(voi['dx']/crop_size[0], voi['dy']/crop_size[1], voi['dz']/crop_size[2]) / 1
    if True:#exceed_ratio > 1:
        img = tr.scale3d(img, [1/exceed_ratio]*3)
        x1 = round(voi['x1'] / exceed_ratio)
        x2 = round(voi['x2'] / exceed_ratio)
        y1 = round(voi['y1'] / exceed_ratio)
        y2 = round(voi['y2'] / exceed_ratio)
        z1 = round(voi['z1'] / exceed_ratio)
        z2 = round(voi['z2'] / exceed_ratio)
    else:
        x1 = voi['x1']
        x2 = voi['x2']
        y1 = voi['y1']
        y2 = voi['y2']
        z1 = voi['z1']
        z2 = voi['z2']
        
    dx = x2 - x1
    dy = y2 - y1
    dz = z2 - z1
        
    xpad = max(crop_size[0] - dx, 0)
    ypad = max(crop_size[1] - dy, 0)
    zpad = max(crop_size[2] - dz, 0)

    side_padding = math.ceil(max(xpad, ypad, zpad) / 2)

    pad_img = []

    for ch in range(img.shape[-1]):
        pad_img.append(np.pad(img[:,:,:,ch], side_padding, 'constant'))

    pad_img = np.stack(pad_img, axis=3)

    x1 += side_padding - math.ceil(xpad/2)
    x2 += side_padding + math.floor(xpad/2)
    y1 += side_padding - math.ceil(ypad/2)
    y2 += side_padding + math.floor(ypad/2)
    z1 += side_padding - math.ceil(zpad/2)
    z2 += side_padding + math.floor(zpad/2)
    
    x1=int(x1)
    x2=int(x2)
    y1=int(y1)
    y2=int(y2)
    z1=int(z1)
    z2=int(z2)

    slice_num = (z1+z2)//2
    img_slice = pad_img[:,:, slice_num, :]
    for ch in range(img_slice.shape[-1]):
        img_slice[:, :, ch] *= 255/np.amax(img_slice[:, :, ch])
    img_slice = np.stack([img_slice, img_slice, img_slice], axis=2)
    
    if draw_bbox:
        img_slice[x1:x2, y2, 2, :] = 255
        img_slice[x1:x2, y2, :2, :] = 0
        
        img_slice[x1:x2, y1, 2, :] = 255
        img_slice[x1:x2, y1, :2, :] = 0
        
        img_slice[x1, y1:y2, 2, :] = 255
        img_slice[x1, y1:y2, :2, :] = 0
        
        img_slice[x2, y1:y2, 2, :] = 255
        img_slice[x2, y1:y2, :2, :] = 0
        
    plot_section_multich(img_slice, [x1,x2,y1,y2], fn, num_ch, save_dir=save_dir+"\\"+voi['cls'], wrong_class=wrong_class, pad=pad)

In [21]:
def plot_section_multich(img_slice, coords, fn, num_ch, save_dir=None, wrong_class=None, pad=15):
    #plt.subplot(121)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        
    ch1 = np.transpose(get_cropped_slice(img_slice[:,:,:,0], coords, pad), (1,0,2))
    ch2 = np.transpose(get_cropped_slice(img_slice[:,:,:,1], coords, pad), (1,0,2))
    
    if num_ch == 2:
        ret = np.empty([ch1.shape[0]*2, ch1.shape[1], 3])
        ret[:ch1.shape[0],:,:] = ch1
        ret[ch1.shape[0]:,:,:] = ch2
        
    elif num_ch == 3:
        ch3 = np.transpose(get_cropped_slice(img_slice[:,:,:,2], coords, pad), (1,0,2))

        ret = np.empty([ch1.shape[0]*3, ch1.shape[1], 3])
        ret[:ch1.shape[0],:,:] = ch1
        ret[ch1.shape[0]:ch1.shape[0]*2,:,:] = ch2
        ret[ch1.shape[0]*2:,:,:] = ch3
        
    else:
        raise ValueError("Invalid num channels")
        
    imsave("%s\\%s (pred %s).png" % (save_dir, fn[:-3], wrong_class), ret)

In [22]:
def get_cropped_slice(img_slice, coords, pad):
    x1 = max(coords[0]-pad, 0)
    x2 = min(coords[1]+pad, img_slice.shape[0])
    y1 = max(coords[2]-pad, 0)
    y2 = min(coords[3]+pad, img_slice.shape[1])
    
    return img_slice[x1:x2, y2:y1:-1]

In [32]:
def condense_cm(y_true, y_pred, cls_mapping):
    simplify_map = {'hcc': 0, 'cyst': 1, 'hemangioma': 1, 'fnh': 1, 'cholangio': 2, 'colorectal': 2}
    y_true_simp = np.array([simplify_map[cls_mapping[y]] for y in y_true])
    y_pred_simp = np.array([simplify_map[cls_mapping[y]] for y in y_pred])
    
    return y_true_simp, y_pred_simp, ['hcc', 'benign', 'malignant non-hcc']

In [24]:
def save_output(Z, y_pred, cls_mapping):
    save_dir = "classified_imgs"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    if not os.path.exists(save_dir + "\\correct"):
        os.makedirs(save_dir + "\\correct")
    else:
        print("images may already exist...")
    if not os.path.exists(save_dir + "\\incorrect"):
        os.makedirs(save_dir + "\\incorrect")

    for i in range(len(Z)):
        if y_pred[i] != y_true[i]:
            plot_multich_with_bbox(Z[i], cls_mapping[y_pred[i]], num_ch=3, pad=30, draw_bbox=True, save_dir=save_dir + "\\incorrect")
        else:
            plot_multich_with_bbox(Z[i], cls_mapping[y_pred[i]], num_ch=3, pad=30, draw_bbox=True, save_dir=save_dir + "\\correct")  

In [29]:
#plot_with_bbox(fn_list[2], cls_mapping[wrong_guesses[2]])
Y_pred = model.predict(X_test)
y_true = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_test])
y_pred = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_pred])

cm = confusion_matrix(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average="weighted")

#save_output(Z_test, y_pred, cls_mapping)

In [26]:
Y_pred = model.predict(X_train_orig)
y_true = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_train_orig])
y_pred = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_pred])

cm = confusion_matrix(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average="weighted")

save_output(Z_train_orig, y_pred, cls_mapping)

images may already exist...


In [146]:
simplify_map = {'hcc': 0, 'cyst': 1, 'hemangioma': 1, 'fnh': 1, 'cholangio': 2, 'colorectal': 2}

In [33]:
y_true_simp, y_pred_simp, _ = condense_cm(y_true, y_pred, cls_mapping)

In [117]:
cls_mapping

['cyst', 'cholangio', 'hcc', 'hemangioma']

In [64]:
cm

array([[26,  3,  1,  3, 13],
       [20, 14,  1,  2, 11],
       [ 3,  3, 14,  2,  8],
       [10,  9,  2, 11, 20],
       [ 1,  2,  0,  1, 20]], dtype=int64)

In [34]:
accuracy_score(y_true_simp, y_pred_simp)

0.86956521739130432

In [30]:
accuracy_score(y_true, y_pred)

0.82608695652173914

def plot_1ch_with_bbox(fn, wrong_class, pad=15, draw_bbox=False, save_dir=None):
    voi_df = pd.read_csv('vois.csv')
    z = re.search(r'^(.*)_(\d*)\.', fn)
    
    img_fn = z.group(1) + ".npy"
    df = voi_df[voi_df["Filename"] == img_fn].iloc[int(z.group(2))]
    
    img = np.load("full_imgs\\"+img_fn)
    #if
    slice_num = (df['z1']+df['z2'])//2
    img_slice = img[:,:, slice_num, 0]
    img_slice = np.stack([img_slice, img_slice, img_slice], axis=2)
    
    if draw_bbox:
        img_slice[df['x1']:df['x2'], df['y2'], 0] = 1
        img_slice[df['x1']:df['x2'], df['y2'], 1:] = 0
        
        img_slice[df['x1']:df['x2'], df['y1'], 0] = 1
        img_slice[df['x1']:df['x2'], df['y1'], 1:] = 0
        
        img_slice[df['x1'], df['y1']:df['y2'], 0] = 1
        img_slice[df['x1'], df['y1']:df['y2'], 1:] = 0
        
        img_slice[df['x2'], df['y1']:df['y2'], 0] = 1
        img_slice[df['x2'], df['y1']:df['y2'], 1:] = 0
        
    if save_dir is None:
        plot_section(img_slice, df, fn, pad=pad)
    else:
        plot_section(img_slice, df, fn, save_dir=save_dir+"\\"+df['cls'], wrong_class=wrong_class, pad=pad, noshow=True)

In [365]:
fn_list = Z_test[~np.equal(y_pred, y_true)]
wrong_guesses = np.array(y_pred)[~np.equal(y_pred, y_true)]

def plot_section(img_slice, row, fn, save_dir=None, wrong_class=None, pad=15, noshow=False):
    #plt.subplot(121)
    if noshow:
        ret = np.transpose(get_cropped_slice(img_slice, row, pad), (1,0,2))
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        imsave("%s\\%s (pred %s).png" % (save_dir, fn[:-3], wrong_class), ret)
        
    else:
        ret = np.transpose(get_cropped_slice(img_slice[:,:,0], row, pad), (1,0))
        plt.imshow(ret, cmap='gray')
    #plt.subplot(122)

In [409]:
Z_test[~np.equal(y_pred, y_true)]

array(['E100815762_0.npy', '12239513_0.npy', '12042703_2.npy'],
      dtype='<U16')

In [380]:
Z_test

array(['E105152534_0.npy', '13166464_0.npy', '12789019_1.npy',
       'E103514998_0.npy', 'E104430882_0.npy', '13064153_0.npy',
       '12857716_0.npy', '13064153_1.npy', 'E100718398_0.npy',
       'E101636006_0.npy', 'E105464882_0.npy', '12239513_0.npy',
       '12229245_0.npy', '12302576_2.npy', '12302576_0.npy'],
      dtype='<U16')

In [159]:
activ = model2.predict(X_train)
#activ = model2.predict(np.expand_dims(X_train[10],axis=0))

In [316]:
import operator
#print("Ground truth:", [cls_mapping[max(enumerate(x), key=operator.itemgetter(1))[0]] for x in Y_val[::30]])
Y_ = model.predict(X_test)
print("Predictions:", [cls_mapping[max(enumerate(x), key=operator.itemgetter(1))[0]] + " (%.5f%% confidence)" % (max(x)*100) for x in Y_])

Predictions: ['hcc (95.75861% confidence)', 'hcc (99.98515% confidence)', 'hcc (56.77554% confidence)', 'hcc (97.68384% confidence)', 'hcc (96.73136% confidence)', 'hcc (99.99979% confidence)', 'hcc (99.99988% confidence)', 'hcc (99.71866% confidence)', 'cyst (63.61439% confidence)', 'cyst (51.41591% confidence)', 'hcc (99.65365% confidence)', 'hcc (52.72004% confidence)', 'hcc (57.44457% confidence)', 'cyst (99.82330% confidence)', 'cyst (83.69393% confidence)']


In [21]:
voi_df_art[voi_df_art["Filename"] == "12673726.npy"]

,Filename,x1,x2,y1,y2,z1,z2,cls,flipz,dx,dy,dz,id
119,12673726.npy,105,134,76,101,40,53,hemangioma,True,29,25,13,126737261057640
120,12673726.npy,127,139,103,117,44,51,hemangioma,True,12,14,7,1267372612710344


In [22]:
voi_df_ven[voi_df_ven["id"] == "1267372612710344"]

,id,x1,x2,y1,y2,z1,z2,dx,dy,dz
74,1267372612710344,127,139,103,117,44,52,12,14,8


In [19]:
max(sum(y)/len(y), 1-sum(y)/len(y))

0.6875

In [86]:
a=time.time()
Y_ = model.predict(X_val)
print(time.time()-a)

0.20054101943969727


In [136]:
import matplotlib.pyplot as plt
%matplotlib inline